<a target="_blank" href="https://colab.research.google.com/github/UpstageAI/cookbook/blob/main/cookbooks/upstage/Solar-Full-Stack LLM-101/05_3_OracleDB.ipynb">
<img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Retrieval Augmented Generation (RAG) Baseline
## Overview  
In this time, we will check the baseline code.
The goal of this project is to provide students with hands-on experience in handling and enhancing Large Language Models (LLMs) provided by [**Upstage**](https://www.upstage.ai) (Solar).

You can use any engineering method for improving benchmark performance excluding direct training (Fine-tuning).

*Collecting data directly related to the test set is considered cheating e.g., using MMLU-pro dataset or EWHA.pdf for KB*

# Baseline

In [ ]:
#!pip3 install -qU python-dotenv PyPDF2 langchain langchain-community langchain-core langchain-text-splitters langchain_upstage oracledb python-dotenv

In [ ]:
# Additional Contents : https://wikidocs.net/253106

In [2]:
# set parameters

file = open("info/api.txt", "r")
api_key = file.read()
file.close()

file = open("info/datapath.txt", "r")
data_path = file.read()
file.close()

file = open("info/resultspath.txt", "r")
results_path = file.read()
file.close()

In [3]:
from langchain_upstage import UpstageEmbeddings

# 쿼리 전용 임베딩 모델
query_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-query")

# 문장 전용 임베딩 모델
passage_embeddings = UpstageEmbeddings(api_key=api_key, model="solar-embedding-1-large-passage")

In [4]:
# funcion to extract an answer from response

import re

def extract_answer(response):
    """
    extracts the answer from the response using a regular expression.
    expected format: "[ANSWER]: (A) convolutional networks"

    if there are any answers formatted like the format, it returns None.
    """
    pattern = r"\[ANSWER\]:\s*\((A|B|C|D|E)\)"  # Regular expression to capture the answer letter and text
    match = re.search(pattern, response)

    if match:
        return match.group(1) # Extract the letter inside parentheses (e.g., A)
    else:
        return extract_again(response)

def extract_again(response):
    pattern = r"\b[A-J]\b(?!.*\b[A-J]\b)"
    match = re.search(pattern, response)
    if match:
        return match.group(0)
    else:
        return None

## 1. build DB

In [5]:
from langchain_upstage import UpstageLayoutAnalysisLoader
import os

UPSTAGE_API_KEY = api_key

layzer = UpstageLayoutAnalysisLoader(api_key=UPSTAGE_API_KEY,file_path=os.path.join(data_path, 'ewha.pdf'), output_type="text")

docs = layzer.load()  # or layzer.lazy_load()

/home/aix7101/anaconda3/envs/yoonnlp/lib/python3.10/site-packages/langchain_upstage/layout_analysis.py:135: UserWarning: UpstageLayoutAnalysisLoader is deprecated.Please use langchain_upstage.document_parse.UpstageDocumentParseLoader instead.
  warnings.warn(
/home/aix7101/anaconda3/envs/yoonnlp/lib/python3.10/site-packages/langchain_upstage/layout_analysis_parsers.py:160: UserWarning: UpstageLayoutAnalysisParser is deprecated.Please use langchain_upstage.document_parse_parsers.UpstageDocumentParseParser instead.
  warnings.warn(


## 2. Text Split

In [6]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)

chunk_size, chunk_overlap = 500, 100

# 2. Split
text_splitter = RecursiveCharacterTextSplitter.from_language(
    chunk_size=chunk_size, chunk_overlap=chunk_overlap, language=Language.HTML
)
splits = text_splitter.split_documents(docs)
print("Splits:", len(splits))

Splits: 119


In [7]:
print(splits[0])
len(splits[0].page_content)

page_content='이화여자대학교 학칙1946. 8. 15. 제정
2017. 8. 16. 개정제1장 총칙제1조(목적) 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과
그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을 도야하여 국가와 인류사회의 발전에
공헌할 수 있는 지도여성을 양성함을 목적으로 한다.제2조(명칭) 본교는 이화여자대학교라 부른다.
제3조(위치) 본교는 서울특별시 서대문구 이화여대길 52에 둔다. (개정 2013.2.25.)제2장 편제제4조(대학 및 대학원) ① 본교에는 다음 각 호의 대학을 둔다.1. 인문과학대학, 사회과학대학, 자연과학대학, 엘텍공과대학, 음악대학, 조형예술대학, 사범
대학, 경영대학, 신산업융합대학, 의과대학, 간호대학, 약학대학, 스크랜튼대학(이하 “각
대학”이라 한다) (개정 2016.6.16.)
2. 호크마(HOKMA)교양대학
② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대
학원, 디자인대학원' metadata={'total_pages': 54}


500

## 3. query-context matching

In [8]:
# read samples.csv file
import pandas as pd

def read_data(data_path):
    data = pd.read_csv(data_path)
    prompts = data['prompts']
    answers = data['answers']
    # returns two lists: prompts and answers
    return prompts, answers

In [9]:
prompts, answers = read_data(os.path.join(data_path, 'test_own_ewha.csv'))

In [10]:
testdata = pd.read_csv(data_path+'test_own_ewha.csv')

In [38]:
nowtest = pd.DataFrame(columns=['index', 'embed_ques', 'question', 'prompts', 'answers', 'top1', 'top2', 'top3', 'predict', 'ground' ])

for index, row in testdata.iterrows():
    #if index % 100 != 0 : continue # 일단 실험할 땐 100개 단위로 끊어서 가져옴
    q = row.prompts
    a = row.answers
    question = q.partition('(A)')[0]
    question = question.partition(') ')[2]
    q = q.partition(') ')[2]
    try : 
        embedded_query = query_embeddings.embed_query(question) # 질문만 받아와서 embedding 하기
        nowtest.loc[len(nowtest)] = {'index':index, 'embed_ques' : embedded_query, 'question' : question, 'prompts' : q, 'answers' : a, 'top1': None, 'top2': None, 'top3': None, 'predict': None, 'gound': None }

    except :
        print(f'pass: {index}')
        continue 


In [39]:
nowtest

,index,embed_ques,question,prompts,answers,top1,top2,top3,predict,ground
0,0,"[-0.005329132080078125, 0.0017337799072265625,...",영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,영어 및 정보 등에 관하여 일정한 기준의 능력이나 자격을 취득한 경우 인정 받는 학...,(D),None,None,None,None,NaN
1,1,"[-0.00460052490234375, -0.0174713134765625, -0...",각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n,각 대학에 따른 학위의 종류로 해당하지 않는 것은 무엇인가?\n(A) 이학사\n(B...,(E),None,None,None,None,NaN
2,2,"[0.00745391845703125, -0.0217132568359375, 0.0...",부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n,부칙에 학과명 변경 및 폐과된 학과는 무엇에 의해 언제까지 존속하는가?\n(A) 현...,(B),None,None,None,None,NaN
3,3,"[-0.002162933349609375, -0.039215087890625, -0...",학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n,학생 포털에서 본인의 성적을 조회할 수 있는 시기는 언제입니까?\n(A) 시험 직후...,(B),None,None,None,None,NaN
4,4,"[0.0089111328125, -0.0259552001953125, -0.0064...",학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,학칙 제1조(목적)에 따르면 이화여자대학교의 설립 목적에 해당하지 않는 것은 무엇입...,(C),None,None,None,None,NaN
5,5,"[0.0016117095947265625, -0.019683837890625, -0...",학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n,학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소는 무엇입니까?\n(A) 서...,(D),None,None,None,None,NaN
6,6,"[-0.018157958984375, -0.0157318115234375, -0.0...",학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?\n,학칙 제12조(휴업일)에서 임시휴업을 결정할 수 있는 근거로 인정되지 않는 경우는?...,(C),None,None,None,None,NaN
7,7,"[0.0029163360595703125, -0.018707275390625, -0...",학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,학칙 제23조(수업연한)에 따르면 건축학과와 약학대학의 수업연한이 다른 이유로 올바...,(D),None,None,None,None,NaN
8,8,"[-5.4895877838134766e-05, -0.01477813720703125...",학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,학칙 제26조(휴학)에서 규정된 휴학의 최대 연장 가능 기간을 초과하는 경우가 아닌...,(C),None,None,None,None,NaN
9,9,"[-0.002269744873046875, 0.0017795562744140625,...",학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n,학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n(A...,(B),None,None,None,None,NaN


## 4. Prompt engineering

In [147]:
del [[wrongdf]]
wrongdf = pd.DataFrame(columns=['questionNum', 'answer', '1st_pred', '2nd_pred'])

In [148]:
############# first PREDICTION ##########

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage

 
llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    [Question]에 대한 정답을 출력해. 주어진 [Context]에서 관련된 정보를 찾아.
    
    정확하게 정답만 답변해.
    그리고 답변이 정확한지 한 번 더 확인해
    ---
    
    [Question] {question}
    [Context] {context}
    [Answer]
        
    '''

)
ko_chain = prompt_template | llm


In [149]:
import numpy as np

chunk = []
for index in range(0,len(splits)) : # context 받아오기
    chunk.append(splits[index].page_content)

embedded_documents = passage_embeddings.embed_documents(chunk)

responses = []
for idx, row in nowtest.iterrows() : # 질문 받아오기 

    embed_ques= row.embed_ques

    # 유사도 기준 내림차순 정렬
    sorted_idx = (np.array(embed_ques) @ np.array(embedded_documents).T).argsort()[::-1]

    nowtest.loc[idx, 'top1'] = chunk[sorted_idx[0]]
    nowtest.loc[idx, 'top2'] = chunk[sorted_idx[1]]
    nowtest.loc[idx, 'top3'] = chunk[sorted_idx[2]]

    response = ko_chain.invoke({"question": row.prompts, "context": chunk[sorted_idx[0]]}) # top1에 대해서는 바로 답변 출력
    responses.append(response.content)

    nowtest.loc[idx, 'predict'] = response.content

responses



['(D) 3학점',
 '(E) 인공지능학사',
 '(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다',
 '(B) 성적 입력 마감 후',
 '(D) 학술의 응용방법 연구',
 '(D) 강남구',
 '(C) 학생총회 요청',
 '(B) 약학대학은 국제 인증 기준에 따라 6년이 요구된다.',
 '(E) 중대한 질병으로 인한 휴학: 총장의 판단에 따라 연장 가능',
 '(B) 전공별 종합시험 합격',
 '(B) 과제 제출 기한 초과',
 '(C) 독립된 학과로 운영된다.',
 '정답은 (D) 타교에 입학한 경우입니다.\n\n이유는 다음과 같습니다:\n\n제시된 맥락에서는 [Question]에 대한 정답을 찾기 위해 학칙 제28조(제적)를 참고해야 합니다. 학칙 제28조(제적)에 따르면, 다음 각 호의 1에 해당하는 학생은 총장이 이를 제적한다고 명시되어 있습니다.\n\n학칙 제28조(제적)\n1. 휴학기간 경과 후 3주일 이내에 정당한 이유 없이 복학하지 아니한 자\n2. 삭제 (2017.8.16.)\n3. 수업료 기타 납입금을 소정 기일 내에 납입하지 아니한 자\n4. 성적이 불량한 자 (개정 2002.11.8)\n5. 재학연한 내에 본교 소정 전과정을 이수하지 못한 자 (개정 2014.11.21.)\n6. 타교에 입학한 자\n7. 삭제 (2003.2.7)\n8. 징계에 의하여 퇴학처분을 받은 자 (신설 2002.2.14)\n\n제시된 각 선택지를 살펴보면, (D) 타교에 입학한 경우는 학칙 제28조(제적)의 6번 항목에 포함되어 있습니다. 따라서 (D) 타교에 입학한 경우는 제적 사유로 포함되지 않는 경우입니다.',
 '(C) 개교기념일',
 '정답은 (A) 신입생 첫 학기입니다.',
 '정답은 (C) 15시간 이상입니다.',
 '정답은 (E) 특정 종교 교리 확립입니다. \n\n이화여자대학교 학칙 제1조(목적)에 따르면, 본교는 대한민국의 교육이념과 기독교정신을 바탕으로 하여 학술의 깊은 이론과 그 광범하고 정밀한 응용방법을 교수․연구하며, 인격을

In [150]:
nowtest.top1[1]

'라 한다) (개정 2016.6.16.)\n2. 호크마(HOKMA)교양대학\n② 본교에는 대학원, 국제대학원, 통역번역대학원, 경영전문대학원, 법학전문대학원, 교육대\n학원, 디자인대학원, 사회복지대학원, 신학대학원, 정책과학대학원, 공연예술대학원, 임상보\n건융합대학원, 임상치의학대학원, 외국어교육특수대학원을 둔다(이하 “각 대학원”이라 한다).\n(개정 2016.6.16., 2017.5.15.)\n[전문개정 2015.11.27.]제5조(학부․학과․전공 및 정원) ① 각 대학, 학부, 학과, 전공 및 모집단위별 입학정원은 별표\n1과 같다. (개정 2015.5.8., 2016.2.16., 2016.2.26., 2016.5.19., 2017.5.4., 2017.5.15.)② 모집단위별 입학정원의 일부는 입학전형에 따라 2개 이상의 모집단위를 통합하여 모집\n할 수 있다. (개정 1999.2.9., 2017.5.15.)\n③ 제2항에 따라 통합된 모집단위로 입학한 학생과 대학 또는 학부 등 광역화된 모'

In [151]:
######### 정답 1차 확인 + wrong 뽑아내기 ######

# print accuracy

cnt = 0
wrong = []
for idx, (answer, response) in enumerate(zip(answers, nowtest['predict'])):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer == None:
        wrong.append(idx+1)
        wrongdf.loc[len(wrongdf)] = {"questionNum":idx+1, "answer" : answer, "1st_pred":generated_answer, "2nd_pred":"-"}
        continue
    
    if generated_answer in answer:
        cnt += 1
    else : 
        wrong.append(idx+1)
        wrongdf.loc[len(wrongdf)] = {"questionNum":idx+1, "answer" : answer, "1st_pred":generated_answer, "2nd_pred":None}
        
acc = cnt/len(answers)*100
print(f"acc: {acc}%")
print()
print("1st wrong:", wrong)
wrongdf.loc[len(wrongdf)] = {"questionNum":"-", "answer" : "-", "1st_pred":acc, "2nd_pred":None}

----------
(D) 3학점
idx: 0 | generated answer: D, answer: (D)
----------
(E) 인공지능학사
idx: 1 | generated answer: E, answer: (E)
----------
(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다
idx: 2 | generated answer: B, answer: (B)
----------
(B) 성적 입력 마감 후
idx: 3 | generated answer: B, answer: (B)
----------
(D) 학술의 응용방법 연구
idx: 4 | generated answer: D, answer: (C)
----------
(D) 강남구
idx: 5 | generated answer: D, answer:  (D)
----------
(C) 학생총회 요청
idx: 6 | generated answer: C, answer:  (C)
----------
(B) 약학대학은 국제 인증 기준에 따라 6년이 요구된다.
idx: 7 | generated answer: B, answer:  (D)
----------
(E) 중대한 질병으로 인한 휴학: 총장의 판단에 따라 연장 가능
idx: 8 | generated answer: E, answer:  (C)
----------
(B) 전공별 종합시험 합격
idx: 9 | generated answer: B, answer:  (B)
----------
(B) 과제 제출 기한 초과
idx: 10 | generated answer: B, answer:  (B)
----------
(C) 독립된 학과로 운영된다.
idx: 11 | generated answer: C, answer:  (C)
----------
정답은 (D) 타교에 입학한 경우입니다.

이유는 다음과 같습니다:

제시된 맥락에서는 [Question]에 대한 정답을 찾기 위해 학칙 제28조(제적)를 참고해야 합니다. 학칙 제28조(제적)에 따르면, 다음 각 

In [152]:
wrongdf

,questionNum,answer,1st_pred,2nd_pred
0,5,(C),D,None
1,8,(D),B,None
2,9,(C),E,None
3,13,(B),D,None
4,14,(E),C,None
5,20,(C),B,None
6,22,(D),C,None
7,25,(B),A,None
8,-,-,68.0,None


In [153]:
############# Second PREDICTION ##########

from langchain_core.prompts import PromptTemplate
from langchain_upstage import ChatUpstage


llm = ChatUpstage(api_key = api_key)

prompt_template = PromptTemplate.from_template(
    '''
    
    [Question]에 대한 정답을 출력해. 주어진 [Context]에서 관련된 정보를 찾아.
    
    어느 부분에서 정보를 찾았는지 설명해.
    정확한 답변을 도출해.
    ---
    
    [Question] {question}
    [Context] {context} {context}
    [Answer]
    [설명]
        
    '''

)
ko_chain = prompt_template | llm


response2 = []
for idx, row in nowtest.iterrows() : ############### 일단 지금은 100개 단위로 띄엄띄엄 test 중
    if idx not in wrong : continue
        
    response = ko_chain.invoke({"question": row.prompts, "context": row.top1, "context": row.top2}) # 선지 전까지 받아오기
    response2.append(response.content)
    nowtest.loc[idx, 'predict'] = response.content

response2

['[Answer] (D) 강남구\n\n[설명]\n본 질문은 학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소를 묻는 것입니다. 학칙 제3조에는 "본교의 주소는 서울특별시 서대문구 이화여대길 52로 한다."라고 명시되어 있습니다. 따라서, 주어진 주소 요소 중에서 "서울특별시", "서대문구", "이화여대길 52"는 모두 본교의 주소에 포함되어 있습니다. 그러나, "강남구"는 본교의 주소에 포함되어 있지 않으므로, 정답은 (D) 강남구입니다.',
 '답변: (B) 창업으로 인한 휴학: 2년.\n\n설명: 학칙 제26조(휴학)에 따르면, 휴학의 최대 연장 가능 기간은 통산하여 3년(건축학전공의 경우 4년, 의예과의 경우 3학기)으로 규정되어 있습니다. 따라서, 주어진 옵션 중에서 최대 연장 가능 기간을 초과하는 경우는 (B) 창업으로 인한 휴학: 2년입니다.',
 '[Question] 학칙 제50조(졸업)에서 복수전공 이수자가 요구되는 추가 조건은 무엇입니까?\n(A) 졸업요건 학점 충족\n(B) 전공별 종합시험 합격\n(C) 졸업 논문 제출\n(D) 총 평균성적 3.0 이상\n(E) 영어강의 이수\n(F) 훈련학점 취득\n\n[Context]\n2.29.)② 제1항의 규정에 불구하고 총장은 교과과정상의 필요에 따라 특정 학과 또는 전공의 졸업에 필요한 학점을 129학점 이상으로 정할 수 있다. (개정 2012.2.29)\n③ 학생은 재학기간중 제1항 또는 제2항의 학점외에 훈련학점을 취득하여야 한다.\n④ 삭제 (2000.6.20.)\n⑤ 학생은 재학기간 중 제1항 또는 제2항의 학점 내에서 총장이 정하는 소정의 학점을 영어강의로 이수하여야 한다. (신설 2013.11.20)\n⑥ 스크랜튼학부 자유전공 입학생의 졸업에 필요한 학점은 제1전공의 졸업에 필요한 학점으로 한다. (신설 2009.2.23.)\n\n[Answer] (F) 훈련학점 취득\n\n[설명]\n학칙 제50조(졸업)에 따르면, 복수전공 이수자는 졸업에 필요한 학점 외에도 훈련학점을 취득해야 

In [154]:
######### 정답 2차 확인 + wrong 뽑아내기 ######

# print accuracy

cnt = 0
wrong2 = []
for idx, (answer, response) in enumerate(zip(answers, nowtest['predict'])):
    print("-"*10)
    generated_answer = extract_answer(response)
    print(response)
    # check
    if generated_answer:
        print(f"idx: {idx} | generated answer: {generated_answer}, answer: {answer}")
    else:
        print("extraction fail")

    if generated_answer == None:
        wrong2.append(idx+1)
        wrongdf.loc[wrongdf['questionNum']==idx+1, '2nd_pred'] = "-"
        continue
    
    if generated_answer in answer:
        cnt += 1
    else : 
        wrong2.append(idx+1)
        wrongdf.loc[wrongdf['questionNum']==idx+1, '2nd_pred'] = generated_answer
        

acc = cnt/len(answers)*100
print(f"acc: {acc}%")
print()
print("2nd wrong:", wrong2)
wrongdf.loc[len(wrongdf)-1, '2nd_pred'] = acc

----------
(D) 3학점
idx: 0 | generated answer: D, answer: (D)
----------
(E) 인공지능학사
idx: 1 | generated answer: E, answer: (E)
----------
(B) 구 학칙에 의하여 재학생이 졸업하는 연도까지 존속한다
idx: 2 | generated answer: B, answer: (B)
----------
(B) 성적 입력 마감 후
idx: 3 | generated answer: B, answer: (B)
----------
(D) 학술의 응용방법 연구
idx: 4 | generated answer: D, answer: (C)
----------
[Answer] (D) 강남구

[설명]
본 질문은 학칙 제3조에서 명시된 본교의 주소에 포함되지 않는 요소를 묻는 것입니다. 학칙 제3조에는 "본교의 주소는 서울특별시 서대문구 이화여대길 52로 한다."라고 명시되어 있습니다. 따라서, 주어진 주소 요소 중에서 "서울특별시", "서대문구", "이화여대길 52"는 모두 본교의 주소에 포함되어 있습니다. 그러나, "강남구"는 본교의 주소에 포함되어 있지 않으므로, 정답은 (D) 강남구입니다.
idx: 5 | generated answer: D, answer:  (D)
----------
(C) 학생총회 요청
idx: 6 | generated answer: C, answer:  (C)
----------
(B) 약학대학은 국제 인증 기준에 따라 6년이 요구된다.
idx: 7 | generated answer: B, answer:  (D)
----------
답변: (B) 창업으로 인한 휴학: 2년.

설명: 학칙 제26조(휴학)에 따르면, 휴학의 최대 연장 가능 기간은 통산하여 3년(건축학전공의 경우 4년, 의예과의 경우 3학기)으로 규정되어 있습니다. 따라서, 주어진 옵션 중에서 최대 연장 가능 기간을 초과하는 경우는 (B) 창업으로 인한 휴학: 2년입니다.
idx: 8 | gen

In [155]:
wrongdf.set_index(keys=['questionNum'])
wrongdf

,questionNum,answer,1st_pred,2nd_pred
0,5,(C),D,D
1,8,(D),B,B
2,9,(C),E,B
3,13,(B),D,D
4,14,(E),C,C
5,20,(C),B,B
6,22,(D),C,C
7,25,(B),A,A
8,-,-,68.0,64.0


## groundedness check

### 질문할 때마다 답이 달라져서 안 할 거임 

In [ ]:
import os
from langchain_upstage import UpstageGroundednessCheck # langchain_upstage==0.1.3
 
groundedness_check = UpstageGroundednessCheck()
 
request_input = {
    "context": f"Mauna Kea is an inactive volcano on the island of Hawai'i. Its peak is 4,207.3 m above sea level, making it the highest point in Hawaii and second-highest peak of an island on Earth.",
    "answer": "Mauna Kea is 5,207.3 meters tall.",
}
response = groundedness_check.invoke(request_input)
print(response)